In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
import random

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Procesamiento del lenguaje natural - CEIA - 2025
## Trabajo practico 1
### Ceballos,Luciano
### a2110

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Carga de datos

In [32]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [33]:
# instanciamos un vectorizador
tfidfvect = TfidfVectorizer()

In [34]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

In [38]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
print(f'Nombres de las clases \n{newsgroups_test.target_names}')

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
Nombres de las clases 
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [37]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

----
## Respuestas:

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [40]:
# Generar 5 índices aleatorios
random_indices = [random.randint(0, X_train.shape[0] - 1) for _ in range(5)]
print("Indices generados: ",random_indices)

Indices generados:  [9727, 4377, 4641, 339, 2576]


In [51]:
for idx in random_indices:
  print('---*3')
  print('\nNuevo documento:\n')
  print(f'El documento seleccionado está en la posicion {idx}: \n')
  print(f'El documento es: \n{newsgroups_train.data[idx]}')
  print('\nPertenece a la clase ',newsgroups_train.target_names[y_train[idx]])
  # Similaridad coseno con todos los documentos de train
  cossim = cosine_similarity(X_train[idx], X_train)[0]
  # los 5 documentos más similares:
  mostsim = np.argsort(cossim)[::-1][1:6]
  # y los 5 más similares son de las clases:
  print("\nLos 5 documentos mas similares son: ",mostsim)
  for i in mostsim:
    print(f'\n\tDocumento {i}: \n{newsgroups_train.data[i]}')
    print(f'\n\tClase: \n{newsgroups_train.target_names[y_train[i]]}\n')


---*3

Nuevo documento:

El documento seleccionado está en la posicion 9727: 

El documento es: 

I'm sure zero-intested economical systems survive on a small-scale,
co-ops is not an Islamic invention, and we have co-operatives working
all around the world. However such systems don't stand the corruption
of a large scale operation. Actually, nothing could handle human
greed, IMHO. Not even Allah :-).

Cheers,
Kent

Pertenece a la clase  alt.atheism

Los 5 documentos mas similares son:  [10448  2364  9459  1909 10878]

	Documento 10448: 

Looking at historical evidence such 'perfect utopian' islamic states
didn't survive. I agree, people are people, and even if you might
start an Islamic revolution and create this perfect state, it takes 
some time and the internal corruption will destroy the ground rules --
again.

Cheers,
Kent

	Clase: 
alt.atheism


	Documento 2364: 

Hehehe, so you say, but this objective morality somehere tells you 
that this is not the case, and you don't know all

**Conclusiones de los resultados obtenidos:**
- Primer documento seleccionado (posición 9727): Pertenece a la categoría atheism y trata sobre los límites prácticos de los sistemas económicos idealistas cuando se enfrentan a la realidad humana.Los documentos encontrados como similares pertenecen todos a la misma clase, excepto el 4 mas similar que pertenece a la categoría Windows.x.
Los documentos similares tratan temas relacionados a los estados "utopicos", corrupcion, moral, la religión islamica desde una perspectiva crítica de alguien no creyente. El documento que está eiquetado con la categoía Windows.x es el unico que no tiene la misma temática que los otros textos y solo encuentro en común una palabra significativa entre ambos documentos: Scale, quizas esa sea la razón de la similitud.
- Segundo documento seleccionado (posición 4377):  Pertenece a la categoría comp.windows.x y trata sobre preferencias en el dseño de interfaces gráficas, especialmente en el uso de gestores de ventanas en sistemas tipo Unix/Linux. Los documentos seleccionados como similares pertenecen todos a la misma categoría y tratan todos de tematicas similares, considero que en este caso los documentos son similares.
- Tercer documento seleccionado (4641): Pertenece a la categoria med y trata sobre el uso de ultrasonido en obstreticia y una discusión sobre quien está calificado para interpretar los resultados. En los documentos seleccionados como similares existen 2 documentos que son muy similares, tratan sobre los mismos topicos pero no tienen similitud en cuanto al topico central con los otros documentos similares que tratan de derechos humanos en el medio oriente.
- Cuarto documento seleccionado (339): Pertenece a la clase comp.os.ms-windows.misc y trata sobre los desafíos de desarrollo de software en múltiples plataformas (específicamente Mac y DOS/Windows) dentro de una red local (LAN), y cómo la falta de herramientas adecuadas para compartir archivos y controlar versiones afecta la productividad del equipo. Los documentos similares son todos parecidos entre sí pero no tratan de topicos o no utilizan un lenguaje similar al documento objetivo, no es buena la similitud.
- Quinto documento seleccionado (posición 2573): Pertenece a la categoria comp.sys.ibm.pc.hardware y trata sobre el desarrollo de un simulador de un microcontrolador con fines educativos; es un mensaje técnico en un foro o un grupo de discusión, donde alguien solicita ayuda o recursos para el desarrollo. Los documentos similares no son buenos, tratan de temas diversos y tienen etiquetas que no son la misma que el documento analizado.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [43]:
def clasificador_baseline(indice_test):
  idxtest = indice_test
  cossim = cosine_similarity(X_test[idxtest], X_train)[0]
  # El documento más similar es:
  mostsim = np.argsort(cossim)[::-1][1]
  categoria_predict = y_train[mostsim]
  return(categoria_predict)


predicciones = [clasificador_baseline(idx) for idx in range(X_test.shape[0])]


In [44]:
f1_score(y_test, predicciones, average='macro')

0.43949389663357213

El valor obtenido no es bueno, sería lo mismo que asignar al azar las categorias.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [45]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
# Definir X_test,y_test,y_pred
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
# Calculo de la metrica
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

In [46]:

# Definir modelos
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

# Parámetros para búsqueda
param_grid = {
    "MultinomialNB": {"alpha": [0.1, 0.5, 1.0]},
    "ComplementNB": {"alpha": [0.1, 0.5, 1.0]}
}

# Evaluar cada modelo
results = {}
for name, model in models.items():
    grid = GridSearchCV(model, param_grid[name], scoring='f1_macro', cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    results[name] = {
        "best_params": grid.best_params_,
        "f1_macro": f1
    }

# Mostrar resultados
for model_name, result in results.items():
    print(f"Modelo: {model_name}")
    print(f"Mejores parámetros: {result['best_params']}")
    print(f"F1-score macro en test: {result['f1_macro']:.4f}\\n")


Modelo: MultinomialNB
Mejores parámetros: {'alpha': 0.1}
F1-score macro en test: 0.6565\n
Modelo: ComplementNB
Mejores parámetros: {'alpha': 0.1}
F1-score macro en test: 0.6954\n


El valor obtenido es significativamente mejorador, utilizar el modelo ComplementNB es la mejor opción.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [ ]:
# Vectorización con hiperparámetros ajustados
tfidfvect_3 = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1,1), sublinear_tf=True, norm='l2',  stop_words='english')

X_train = tfidfvect_3.fit_transform(newsgroups_train.data)

In [ ]:
# Transponer la matriz documento-término para obtener la matriz término-documento
X_terms = X_train.T
# Obtener el vocabulario
terms = np.array(tfidfvect_3.get_feature_names_out())
list(tfidfvect_3.vocabulary_.keys())[:100]

['wondering',
 'enlighten',
 'car',
 'saw',
 'day',
 'door',
 'sports',
 'looked',
 'late',
 '60s',
 'early',
 '70s',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'bumper',
 'separate',
 'rest',
 'body',
 'know',
 'model',
 'engine',
 'specs',
 'years',
 'production',
 'history',
 'info',
 'funky',
 'looking',
 'mail',
 'fair',
 'number',
 'brave',
 'souls',
 'upgraded',
 'si',
 'clock',
 'oscillator',
 'shared',
 'experiences',
 'poll',
 'send',
 'brief',
 'message',
 'detailing',
 'procedure',
 'speed',
 'attained',
 'cpu',
 'rated',
 'add',
 'cards',
 'adapters',
 'heat',
 'sinks',
 'hour',
 'usage',
 'floppy',
 'disk',
 'functionality',
 '800',
 'floppies',
 'especially',
 'requested',
 'summarizing',
 'days',
 'network',
 'knowledge',
 'base',
 'upgrade',
 'haven',
 'answered',
 'thanks',
 'folks',
 'mac',
 'plus',
 'finally',
 'gave',
 'ghost',
 'weekend',
 'starting',
 'life',
 '512k',
 'way',
 '1985',
 'sooo',
 'market',
 'new',
 'machine',
 'bit',
 'soo

In [ ]:
# Elegir palabras manualmente
selected_words = ["picking", "maybe", "bit", "life", "sinks"]


# Encontrar las 5 palabras más similares a cada una
for word in selected_words:
    if word in tfidfvect_3.vocabulary_:
        idx = tfidfvect_3.vocabulary_[word]
        similarities = cosine_similarity(X_terms[idx], X_terms).flatten()
        similar_indices = similarities.argsort()[-6:-1][::-1]  # Tomar las 5 más similares
        similar_words = terms[similar_indices]
        print(f"Palabras más similares a '{word}': {similar_words}")
    else:
        print(f"'{word}' no está en el vocabulario.")

Palabras más similares a 'picking': ['nit' 'beams' 'gms' '1929' 'speedometer']
Palabras más similares a 'maybe': ['precious' 'cameo' 'just' 'know' 'don']
Palabras más similares a 'bit': ['32' '24' 'bits' '16' 'key']
Palabras más similares a 'life': ['people' 'real' 'sex' 'god' 'membership']
Palabras más similares a 'sinks': ['344' 'heat' 'adcom' 'nd' 'incurred']
